!['Salary'](https://kayako.com/wp-content/uploads/2021/01/Curious-case-of-the-customer-support-managers-salary-970x568-1.png)

# Dataset and motivation

This dataset came from an online survey at askamanager.org (https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html). The goal of this survey was to take the mystery out of salaries and allow people to see how much others were actually making. There are 17 variables consisting of: Timestamp How old are you? Industry Job title Additional context on job title Annual salary Other monetary comp Currency Currency - other Additional context on income Country State City Overall years of professional experience Years of experience in field Highest level of education completed Gender Race

This dataset was downloaded from https://docs.google.com/spreadsheets/d/1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw/edit?resourcekey#gid=1625408792. At the time the dataset was downloaded for use in this analysis there were 27680 responses.

# Research Question

How accurately can salary be predicted based on the features in a dataset collected from an online blog? What are the best predictors of salary?

# Literatrue Review

Needs polishing<Br>
Many studies have been done on data collected from a more structured process designed to ensure a good distribution of a population.  Our study looks at mostly freeform data collected from an online blog.  We wanted to see how well a model could predict salary built from a messy dataset.<BR>
    -I thought about listing one or more other studies here with a summary and a link

# Data Cleaning

## Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import re
import us
from city_to_state import city_to_state_dict

%matplotlib inline
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/ajoseph/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/ajoseph/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Load and explore dataset

In [2]:
df = pd.read_csv('Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1 (1).csv')

In [3]:
df

,Timestamp,How old are you?,Industry,Job title,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27676,2/19/2022 16:07:31,45-54,Property or Construction,Senior Project Manager,Owner rep consultant,145000,14500.0,USD,NaN,Guaranteed 10% bonus if our department meets t...,USA,California,San Francisco,11 - 20 years,11 - 20 years,College degree,Woman,Asian or Asian American
27677,2/19/2022 18:17:08,25-34,Insurance,Underwriter,NaN,67000,6700.0,USD,NaN,NaN,USA,Florida,Tampa,8 - 10 years,11 - 20 years,Master's degree,Woman,White
27678,2/19/2022 19:47:05,25-34,"Accounting, Banking & Finance",Vice Presidenti and Corporate Counsel,(in-house attorney),252000,80000.0,USD,NaN,I also receive an equity grant,US,Pennsylvania,Philadelphia,11 - 20 years,11 - 20 years,"Professional degree (MD, JD, etc.)",Woman,White
27679,2/21/2022 19:43:29,25-34,Education (Early Childhood Education),Quality Rating Specialist,Program assessor to conduct quality assessment...,62400,NaN,USD,NaN,NaN,United States,California,San Diego,11 - 20 years,8 - 10 years,College degree,Woman,Asian or Asian American


In [4]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_notebook_iframe()

In [5]:
df.dtypes

Timestamp                                    object
How old are you?                             object
Industry                                     object
Job title                                    object
Additional context on job title              object
Annual salary                                object
Other monetary comp                         float64
Currency                                     object
Currency - other                             object
Additional context on income                 object
Country                                      object
State                                        object
City                                         object
Overall years of professional experience     object
Years of experience in field                 object
Highest level of education completed         object
Gender                                       object
Race                                         object
dtype: object

In [6]:
df.nunique()

Timestamp                                   24899
How old are you?                                7
Industry                                     1199
Job title                                   14168
Additional context on job title              6909
Annual salary                                4246
Other monetary comp                           836
Currency                                       11
Currency - other                              111
Additional context on income                 2951
Country                                       365
State                                         129
City                                         4758
Overall years of professional experience        8
Years of experience in field                    8
Highest level of education completed            6
Gender                                          5
Race                                           48
dtype: int64

## Cleaning Steps

1) remove 'Timestamp' <br>
2) refine 'How old are you?' into categories<br>
3) refine 'Industry' into categories<br>
4) refine 'Job title' into categories<br>
5) remove 'Additional context on job title' <br>
6) combine 'Annual salary' with 'Other monetary comp' and convert to US currency<br>
7) add 'Currency -other' information to 'Currency' and then remove 'Currency -other' and make 'Currency' categorical <br>
8) remove 'Additional context on income' <br>
9) refine Country into categories<br>
10) impute 'State' from 'City' values<br>
11) remove 'City'<br>
12) refine 'Overall years of professional experience' into categories<br>
13) remove 'Years of experience in field'<br>
14) refine 'Highest level of education completed' into categories<br>
15) refine 'Gender' into categories<br>
16) refine 'Race' into categories<br>



#### Remove unnecessary columns

In [7]:
df=df.drop(['Timestamp'], axis=1)
df=df.drop(['Additional context on job title'], axis=1)
df=df.drop(['Additional context on income'], axis=1)

#### 'Age'

In [8]:
df['Age']=df['How old are you?'].astype('category')
df=df.drop(['How old are you?'], axis=1)

#### 'Country' and 'Overall years of professional experience'

With the 'Country' field appearing as a mandatory free-form text entry on the survey, there resulted in more than 360 unique values in this field despite nearly 80% of the dataset entries originating from individuals based in the U.S.. Due to this anomaly, the best approach seemed to be to create a custom dictionary in order to fix more desirable 'Country' titles to those which had misspellings and abbreviations. Many of the rows had 'Country' field occupied by the incorrect country; this was largely due to the fact that a large group of individuals lived in a Country which was different than the location of the base for their employer or government agency. This was corrected by the given information in the 'Country' column and/or the 'City' or 'State' columns which were occupied by American cities/states, 'District of Columbia', or 'Puerto Rico'. Then, once condensed down to about 120 unique values, the countries which only appeared once were taken out of the dataset completely as they didn't seem to fairly represent the manager salary in those specifically countries compared to the U.S. which still had nearly 22,000 entries for itself.

This dataset included 2 fields for experience, one was for overall professional experience and the other was for the number of years in their current respective field. Several of the entries had 'Years of experience in field' larger than 'Overall years of professional experience' so in order to simplify this, both columns were condensed by taking the larger of the 2 fields and renaming the field 'Experience'. Since both of these columns were multiple choice, they each contained several ranges of experience; upon merging, the bottom of each range was chosen to keep the value a single integer for 'Experience'. 

In [9]:
# Create dictionary of mispelled country names to recognize and set each associated Country correctly
global country_names
country_names={'United States':'USA', 'US':'USA', 'United States of America':'USA', 'United States ':'USA', 'UK ':'UK', 'United Kingdom':'UK', 
               'U.S>':'USA', 'USA ':'USA', 'Usa':'USA', 'usa':'USA', 'Uk':'UK', 'USA-- Virgin Islands':'USA', 'Us':'USA', 'US ':'USA', 
               'U.S.A.':'USA', 'U.S.A':'USA', 'U.S. ':'USA', 'United States':'USA', 'United State':'USA', 'United Kingdom ':'UK', 'Scotland ':'UK',
               'England':'UK', 'England/UK':'UK', 'Scotland':'UK', 'England ':'UK', 'uk':'UK', 'Unites States':'USA', 'Uk ':'UK', 
               'UK (Northern Ireland)':'UK', 'England, UK':'UK', 'U.S.':'USA', 'United Kingdom.':'UK', 'United kingdom':'UK', 'U.K. ':'UK',
               'United kingdom ':'UK', 'England, UK.':'UK', 'United Kingdom (England)':'UK', 'Unted States':'USA', 'ENGLAND':'USA', 'Great Britain':'UK',
               'Us ':'USA', 'UXZ':'USA', 'U.K':'UK', 'United states':'USA', 'united states':'USA', 'United States of America ':'USA', 'Wales, UK':'UK',
               'Northern Ireland, United Kingdom':'UK', 'England, United Kingdom ':'UK', 'England, United Kingdom':'UK', 'America':'USA', 'england':'UK',
               'united stated':'USA', 'Wales (United Kingdom)':'UK', 'Us ':'USA', 'UK (England)':'UK', 'uk':'UK', 'Uk ':'UK', 'England, UK':'UK',
               'U.K. (northern England)':'UK', 'us':'USA', 'U.K.':'UK', 'United Kingdomk':'UK', 'Englang':'UK', 'Wales (UK)':'UK', 'Scotland, UK':'UK',
               'United States- Puerto Rico':'USA', 'UK, remote':'UK', 'UK for U.S. company':'USA', 'Canada ': 'Canada', 'canada':'Canada', 'CANADA':'Canada',
               'Canadw':'Canada', 'Can':'Canada', 'Canda':'Canada', 'Puerto Rico ':'USA', 'England, Gb':'UK', 'UK for U.S. company':'USA', 'Canadá':'Canada',
               'United Kindom':'UK', 'ireland':'Ireland', 'Ireland ':'Ireland', 'Japan ':'Japan', 'Sweden ':'Sweden', 'United Arab Emirates ':'UAE', 
               'Unites kingdom ':'UK', 'Northern Ireland ':'UK', 'united kingdom':'UK', 'Wales':'UK', 'Canada, Ottawa, ontario':'Canada', 'Britain ':'UK',
               'FRANCE':'France', 'CANADA ':'Canada', 'spain':'Spain', 'Canad':'Canada', 'Australia ':'Australia', 'australia':'Australia', 
               'Australi':'Australia', 'Csnada':'Canada', 'France ':'France', 'Puerto Rico':'USA', 'Northern Ireland':'UK', 'germany':'Germany',
               'New Zealand Aotearoa':'New Zealand', 'Aotearoa New Zealand':'New Zealand', 'france':'France', 'New zealand':'New Zealand',
               'New Zealand ':'New Zealand', ' New Zealand':'New Zealand', 'Germany ':'Germany', 'Mexico ':'Mexico', 'Great Britain ': 'UK',
               'finland':'Finland', 'Russia ':'Russia', 'NZ':'New Zealand', 'The Netherlands':'Netherlands', 'Netherlands ':'Netherlands',
               'the netherlands':'Netherlands', 'new zealand':'New Zealand', 'INDIA':'India', 'Spain ':'Spain', 'The netherlands': 'Netherlands', 
               'South Korea ':'South Korea', 'Ukraine ':'Ukraine', 'hong konh':'Hong Kong', 'Hong Kong ':'Hong Kong', 'switzerland':'Switzerland',
               'Brazil ':'Brazil', 'SWITZERLAND':'Switzerland', 'japan':'Japan', 'Australian ':'Australia', 'The Netherlands ':'Netherlands',
               'netherlands':'Netherlands', 'South Africa ':'South Africa', 'Croatia ':'Croatia', 'Czech republic':'Czech Republic', 
               'czech republic':'Czech Republic', 'Czech Republic ':'Czech Republic', 'Italy (South)':'Italy', 'Bangladesh ':'Bangladesh', 
               'singapore':'Singapore', 'Malaysia ':'Malaysia', 'Singapore ':'Singapore', 'Denmark ':'Denmark', 'denmark':'Denmark', 'Norway ':'Norway',
               'Japan, US Gov position':'USA','Remote (philippines)':'USA', 'croatia':'Croatia', 'Company in Germany. I work from Pakistan.':'Germany', 
               'ARGENTINA BUT MY ORG IS IN THAILAND':'Thailand', 'Czechia':'Czech Republic', 'Jersey, Channel islands':'UK', 'Mainland China':'China',
               'Switzerland ': 'Switzerland', 'South africa':'South Africa', 'México':'Mexico', 'Catalonia':'Spain', 'Panamá':'Panama', 'the Netherlands':'Netherlands',
               'Nederland':'Netherlands', 'Brasil':'Brazil', 'Portugal ':'Portugal', 'Sri lanka':'Sri Lanka', 'The Bahamas ':'The Bahamas', 'NL':'Netherlands',
               'europe':'Czech Republic', 'Belgium ':'Belgium'}

In [10]:
def clean_columns(df):

    
    #Condense experience down to only 1 column
    k = df['Overall years of professional experience'].unique()
    exp = {}
    for carrot in k:
        exp[carrot] = 0
    exp['2 - 4 years'] = 2
    exp['5-7 years'] = 5
    exp['8 - 10 years'] = 8
    exp['11 - 20 years'] = 11
    exp['21 - 30 years'] = 21
    exp['31 - 40 years'] = 31
    exp['41 years or more'] = 41
    df['Overall years of professional experience'] = df['Overall years of professional experience'].apply(lambda x: exp.get(x))
    df['Years of experience in field'] = df['Years of experience in field'].apply(lambda x: exp.get(x))
    
    # Correct years of experience if field experience is greater than professional experience and condense both columns into one
    df['Experience'] = df['Overall years of professional experience']
    df.loc[df['Overall years of professional experience'] < df['Years of experience in field'], 'Experience'] = df['Years of experience in field']    
    df = df.drop(['Overall years of professional experience'], axis=1)
    df = df.drop(['Years of experience in field'], axis=1)
    
    # Create list of states to recognize and set each associated Country as 'USA'
    statesL = ['alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida', 'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska', 'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'washington', 'west virginia', 'wisconsin', 'wyoming', 'puerto rico', 'district of columbia']
    states=[]
    for i in statesL:
        new = i.title()
        if new == 'District Of Columbia':
            new = 'District of Columbia'
        states.append(new)
    df.loc[((df.State.isin(states)) | (df.City.isin(states)) | (df.State.isin(city_to_state_dict.keys()))), 'Country'] = 'USA'
    
    df['Country'].replace(country_names, inplace=True)
    country1=df['Country'].value_counts().tail(49).keys().tolist()
    df=df[~df['Country'].isin(country1)]
    return df

df = clean_columns(df)


#### 'Industry'

Because of the wide varity of industries and the element of freeform entry, we wanted to use natural language processing to group them up for analysis.

In [11]:
industry_dict = {
    'education': ['education','instructional','interpretation', 'coach',  'print','higher','primary', 'restoration','secondary', 'educational', 'early','graduate', 'childhood','college','publish', 'edtech','academic', 'research','literature', 'university','teacher','museum','study', 'learn','museums','publications','translation','instructor','arts' ,'university','student','learn','school','archive','textbook','educ','publishibg'],
    'tech':['compute', 'internet','tech','digital', 'ecommerce','aerospace', 'virtual','technology', 'software','saas','cybersecurity','technical','database','information','fintech','data','analytics','intelligence'],
    'finance':['account' ,'fund','actuarial','bank', 'invest','finance','investment','economics','wealth'],
    'nonprofit':['nonprofits','diversity', 'equity', 'inclusion', 'charity','grantwriting','americorps', 'philanthropy', 'donor', 'nonprofit','heritage','culture','foundation','cultural','preservation','fundraise' ],
    'government':[ 'government','disability','global','urban','immigration','regulatory', 'public', 'administration','library','social', 'survey','libraries','politics','relations', 'lobby','librarian','international','govt','community','policy','federal','state','political','mail','management','compliance'],
    'healthcare':['health', 'probiotics','care', 'medcomms','medical','hospital','biopharma','pharmaceuticals','medicine','pharma','pharmaceutical','healthcare','clinical','preclinical','biomedical', 'pharmacy','toxicology','veterinarian','therapy', 'psychology','veterinary','therapist','cancer','drug','psychologist','hygiene'],
    'property':['utilities','appraisal', 'electrification', 'property','concrete','compliance','construction','costruction', 'architecture','real', 'estate' ,'interior' ,'architecture','house','furniture','architect','mortgage','interior','landscape'],
    'business':['business','media','digital','market','advertise', 'design','purchase','buyer',  'survey','review','fmcg','strategy','comm','outsource','payment','center','private','think','commerce','award', 'work','procurement','staff', 'background','human','association','contract','consult', 'insurance','trade','sales','logistics', 'recruitment','service','consumer','management','distribution','workforce','market','wholesale','organizational','labor','executive', 'professional','corporate','procurement','operations','product','administrative','legal','train','union','communications','unions','office','counsel','consultant','consultants','organization', 'logistics',  'compliance',  'plan', 'assist','accessibility'],
    'manufacturing':['engineer','inspection','destructive','quality', 'manufacture', 'distribute', 'production','supply','warehouse','distribution','chemical','chemicals', 'operations','industrial','manufactures','machinery','equipment' ],
    'science':['biotechnology','bioinformatics','aerospace', 'stem', 'research','biotech','space','sciences', 'science','ecology','scientific','scientist','chemistry','biological','bioscience','bitech','laboratory','analytical','biologist','animal'],
    'environment':['energy','exploration', 'park', 'zoos', 'environmental', 'conservation','regulation','archaeology','archaeologist','resource','renewable', 'sustainability','renewables','environment','natural'],
    'agriculture':['agriculture','maritime', 'forestry','mine','cannabis','geospatial','land','geologist','outdoor','horticulture','petroleum','fisherman','oilfield'],
    'law enforcement and military':['enforcement', 'protective','security','soldier','defense','protection','forensics','military'],
    'transportation':['auto','automotive','repair','truck', 'aviation','transport','transportation','airline'],
    'entertainment':['music','perform','entertainment','animation','leisure','sport','video','game','journalism','fitness','theatre','theater','film','music','video','perform'],
    'self-employed':[ 'gamble','retire','entrepreneur','travel','self'],
    'service':['food','storage','language','clothe','facilities', 'maintenance', 'clean','foodservice','beverage','drink', 'beer','beauty','cosmetology','housekeeper','cook','landscape','funeral','retail','hospitality','tourism','restaurant','restaurants','apparel','laundry','wine','brew','waste','customer','fashion','tailor','gyms','auction','camp','pest','coffee','janitorial','grocery','groom'],
    'religion':['religion','church', 'ministry','religious','faith', 'spirituality','clergy','chaplain'],
    'family':['childcare','daycare','children','household','parent','child','caregiver','caretaker','family','sitter']
}

In [12]:
def lemmatize_stemming(text):
    return (WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [13]:
df['Industry']=df['Industry'].astype(str)
processed_text = df['Industry'].map(preprocess)
df['words']=processed_text
topic=[]
for industry in df['words']:
    for word in industry:
        indust=[key for key, value in industry_dict.items() if word in value]

    topic.append(indust)
df['topic']=topic

In [14]:
indust=[]
for line in df['topic']:
    if len(line)==0:
        indust.append(np.nan)
    else:
        try:
            indust.append(','.join(map(str, line)))
        except TypeError:
            indust.append(np.nan)
df=df.drop(['Industry','words'], axis=1)
df=df.drop(['topic'], axis=1)
df['Industry']=indust

#### 'Job title'

Job title is another feature that really presented a challenge for analysis due to the wide variety of industries, freeform text entry, and description of job.  To clean this, we decided that having set bins to group major job titles to categorize would be the best way to compare jobs across industries.  We were able to use key words in the text to properly classify each title into a category.

In [15]:
#binning job titles
df.loc[(df['Job title'].str.contains('chief',case=False)), 'Job title_Bin'] = 'Executive'
df.loc[(df['Job title'].str.contains('\s+c[a-z]o\s+|^c[a-z]o$|\s+c[a-z]o$|^c[a-z]o\/|^c[a-z]o\s',case=False)), 'Job title_Bin'] = 'Executive'
df.loc[(df['Job title'].str.contains('director',case=False)), 'Job title_Bin'] = 'Director'
df.loc[(df['Job title'].str.contains('manag.',case=False)), 'Job title_Bin'] = 'Management'
df.loc[df['Job title_Bin'].isnull(),'Job title_Bin']='Non-Managerial Employee'
df=df.drop(['Job title'], axis=1)

#### 'Salary' and 'Currency'

Many of the salary values did not fit the expected form of 'annual salary'.  Digging into these values, it appeared that they were entered as thousands of dollars instead of actual dollar amount.  We were able to correct these by multiplying by 1,000.  We also wanted to remove the additional compensation column since it was mostly null and combined it with annual salary to get the total annual salary. <Br>
    Since our dataset included the US and other countries, we wanted to normalize the data for a true apples-to-apples comparison, so we converted all the non-US currencies to USD by way of a conversion table given the current exchange rates.

In [16]:
df['Annual salary']=df['Annual salary'].astype('str')
df['Annual salary']=df['Annual salary'].str.replace(',','')
df['Annual salary']=df['Annual salary'].astype('float64')
df['Annual salary']=df['Annual salary'].fillna(0)
corrected=[]
for val in df['Annual salary']:
    if val<100:
        corrected.append(val)
    elif 100<val<1000:
        corrected.append(val*1000)
    else:
        corrected.append(val)
        
df['Other monetary comp']=df['Other monetary comp'].fillna(0)
df["Total Salary"] = corrected + df['Other monetary comp']
df=df.drop(['Annual salary', 'Other monetary comp'], axis=1)

In [17]:
df["Currency"]=df["Currency"].astype(str)
df["Currency - other"]=df["Currency - other"].astype(str)
currency_unique=df['Currency'].unique()

In [18]:
pattern = re.compile(r"(?<![A-Z])[A-Z]{3}(?![A-Z])")

curr=[]
for i in range(len(df["Currency"])):
    val=df["Currency"].iloc[i]
    if val == 'Other':
        result = pattern.search(df['Currency - other'].iloc[i])
        try:
            curr.append(result.group())
        except AttributeError:
            curr.append(np.nan)
    else:
        curr.append(val)

In [19]:
df['Currency']=curr
df.loc[df['Currency'] == "AUD", 'Currency'] = 'AUD/NZD'
df.loc[df['Currency'] == "NZD", 'Currency'] = 'AUD/NZD'

cur=[]
for val in df["Currency"]:
    if val in currency_unique:
        cur.append(val)
    else:
        cur.append(np.nan)
df['Currency']=cur

df = df[df['Currency'].notna()]
df=df.drop(['Currency - other'], axis=1)

In [20]:
exchange ={1:'USD',
           1.3210938:'GBP',
           0.78664093:'CAD',
           1.0873256:'EUR',
           0.74139988:'AUD/NZD',
           1.0861636:'CHF',
           0.06495109:'ZAR',
           0.10061237:'SEK',
           0.12794878:'HKD',
           0.0087003299:'JPY'
          }

In [21]:
convert=[]
for val in df['Currency']:
    convert.append([key for key, value in exchange.items() if val in value][0])
    
df['convert']=convert
df['convert']=df['convert'].astype(float)
df["Total Salary"]=df["Total Salary"].astype(float)
df["Salary (USD)"] = df["convert"] * df["Total Salary"]
df=df.drop(['convert','Total Salary','Currency'], axis=1)

#### 'City' and 'State'

There were many US States that were null but had an entry for City.  We were able to impute many of the state values from looking at state entered in city, city name from a city-state dictionary reference, and getting the state name from the two letter abbreviation through the us package.

In [22]:
def return_state(txt=None):
    try:
        regex = '\s[a-zA-Z]{2}$'
        q=re.search(regex,txt)
        if q==None:
            regex = '^[a-zA-Z]{2}$'
            q=re.search(regex,txt)
            if q==None:
                return np.nan
        st=us.states.lookup(q.group().strip())
        return st.name
    except:
        return np.nan

In [23]:
def cityState(city):
    try:
        return city_to_state_dict[city.title().strip()]
    except:
        return np.nan

In [24]:
#Assigns a value to state for non-us countries
df.loc[(df['Country'] != 'USA'), 'State'] = 'Outside USA'

In [25]:
subParts=['no','na','n/a']
for part in subParts:
    df.loc[(df['Country'] == 'USA') & (df['State'].isnull()) & (df["City"].str.lower() == part), "City"] = 'delete'
df.loc[(df['Country'] == 'USA') & (df['State'].isnull()) & (df["City"].isnull()),'City'] = 'delete'

In [26]:
junk = [   
    'decline',
    'prefer',
    'answer',
    'various ',
    'area',
    'remote',
    'confidential',
    'sorry',
    'metro',
    'coast',
    'several',
    'anonymous',
    'region',
    'new england',
    'home'
    # 'district',
    # 'dc',
    # 'd.c.'
       ]
for j in junk:
    df.loc[(df['Country'] == 'USA') & (df['State'].isnull()) & (df['City'].str.contains(j,case=False)), 'City'] = 'delete'

In [27]:
#populates state from two letter abbreviation in city
df.loc[(df['Country'] == 'USA') & (df['State'].isnull()), 'State']=df.loc[(df['Country'] == 'USA') & (df['State'].isnull()), 'City'].apply(lambda x: return_state(x))

In [28]:
#populates state fromm state name in city
df.loc[(df['Country'] == 'USA') & (df['State'].isnull()), 'State']=df.loc[(df['Country'] == 'USA') & (df['State'].isnull()), 'City'].apply(lambda x: us.states.lookup(x.title().strip()))

In [29]:
#poulates state from city_to_state_dict lookup
df.loc[(df['Country'] == 'USA') & (df['State'].isnull()), 'State']=df.loc[(df['Country'] == 'USA') & (df['State'].isnull()), 'City'].apply(lambda x: cityState(x))

In [30]:
#replaces empty US States with ',' for specific row deletion
df.loc[(df['Country'] == 'USA') & (df['State'].isnull()), 'State']=','

In [31]:
#removes empty US states and multiple selected states
df=df[(df['Country'] == 'USA') & (df['State'].str.contains(','))==False]

In [32]:
df=df.drop(['City'], axis=1)

In [33]:
print(df.shape)

(27336, 10)


#### 'Education'

With no way to impute education from any of the other available columns, we decided to drop the 203 null values since they were such a small percentage of our total record count.

In [34]:
#Drops 203 null rows from 'Highest level of education completed'
df.dropna(subset=['Highest level of education completed'], inplace=True)

#### 'Gender'

Gender for "other" with a freeform entry or "non-binary".  We decided to reduce these options down to "Man", "Woman", and "Other" for ease of analysis without losing any data points.

In [35]:
#Bins Gender column to Man, Woman, and Other
df['Gender_Bin'] = np.where(df['Gender'].isin(['Man', 'Woman']),
                           df['Gender'],'Other')
df=df.drop(['Gender'], axis=1)

#### 'Race'

Race allowed for checkbox selection entry which aided in the cleanliness of the specific values but made it challenging when more than one race was selected.  We were able to work with this by grouping all the multiple selected races into a group called "Mixed Race".  This again allowed us to categorize the variable without losing data points.

In [36]:
#Bins Race column to single selection and groups multi select as Mixed Race
df['Race_Bin'] = np.where(df['Race'].isin(['White',
                                           'Asian or Asian American',
                                           'Hispanic, Latino, or Spanish origin',
                                           'Middle Eastern or Northern African',
                                           'Black or African American',
                                           'Native American or Alaska Native']),
                                           df['Race'],'Mixed Race')
df=df.drop(['Race'], axis=1)

#### Remove any remaining NA rows

The remaning records with a null value in any of the columns were finally dropped to produce a clean dataset for analysis.  Our whole cleaning process only removed 

In [37]:
df=df.dropna(axis=0)

### Re-order columns

In [38]:
cols = list(df.columns.values)
cols

['Country',
 'State',
 'Highest level of education completed',
 'Age',
 'Experience',
 'Industry',
 'Job title_Bin',
 'Salary (USD)',
 'Gender_Bin',
 'Race_Bin']

In [39]:
df = df[['Salary (USD)','Job title_Bin','Industry', 'Country', 'Race_Bin','Age', 'Gender_Bin',
         'Highest level of education completed','Experience']]

###  Export cleaned dataframe to CSV

In [40]:
df.to_csv('Salary.csv',index=False)